In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, message="You are using `torch.load` with `weights_only=False`")
warnings.filterwarnings("ignore", category=UserWarning, message="The PyTorch API of nested tensors is in prototype stage and will change in the near future.")

%load_ext autoreload
%autoreload 2

import sys, os
sys.path.insert(0,"/root/gurusmart/MorganFP_prediction/reproduce_previous_works/Spectre")
            
import torch
torch.set_printoptions(precision=10)


import yaml
torch.set_float32_matmul_precision('high')
from pathlib import Path

from rdkit import Chem
from rdkit.Chem import Draw
# load model 
from datasets.dataset_utils import  fp_loader_configer

fp_loader_configer.select_version("Morgan_FP")
fp_loader = fp_loader_configer.fp_loader

import numpy as np 
import random
seed=2
torch.cuda.manual_seed_all(seed) 
np.random.seed(seed)
random.seed(seed)



choosing Morgan_FP_loader


In [2]:
import pickle
with open(f'/root/gurusmart/MorganFP_prediction/inference_data/coconut_loutus_hyun_training/inference_metadata_latest_RDkit.pkl', 'rb') as file:
    smiles_and_names = pickle.load(file)
print(len(smiles_and_names))

526316


In [3]:
from models.optional_input_ranked_transformer import OptionalInputRankedTransformer
from datasets.optional_2d_folder_dataset import OptionalInputDataModule



In [4]:



loader_idx_to_name_mapping = {
    0: "eHSQC, H NMR, and C NMR",
    1: "eHSQC and H NMR",
    2: "eHSQC and C NMR",
    3: "eHSQC Only",
    4: "C NMR and H NMR",
    5: "H NMR Only",
    6: "C NMR Only",
    7: "Normal HSQC only",
    8: "standard HSQC, H NMR, and C NMR",
    9: "standard HSQC and H NMR",
    10: "standard HSQC and C NMR",
    
    
}

In [5]:
max_radius = 2
fp_dim = 2048
rankingset_path = f"/root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/vanilla_fp_rankingset_max_radius_{max_radius}_fp_dim_{fp_dim}_stacked_together_sparse/FP.pt"

rankingset_data = torch.load(rankingset_path).to("cuda")

/opt/conda/lib/python3.11/site-packages/torch/_utils.py:315: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  result = torch.sparse_compressed_tensor(


In [6]:

def test_extra_input_types(checkpoint_path):
    # checkpoint_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r0_r6_trial_1/checkpoints/epoch=95-step=21696.ckpt"
    checkpoint_path = Path(checkpoint_path)
    model_path = checkpoint_path.parents[1]
    hyperpaerameters_path = model_path / "hparams.yaml"

    with open(hyperpaerameters_path, 'r') as file:
        hparams = yaml.safe_load(file)
        
    del hparams['checkpoint_path'] # prevent double defition of checkpoint_path

    fp_loader = fp_loader_configer.fp_loader
    hparams['num_workers'] = 0
    model = OptionalInputRankedTransformer.load_from_checkpoint(checkpoint_path, fp_loader = fp_loader,  **hparams)
    max_radius = int(hparams['FP_choice'].split("_")[-1])
    fp_loader.setup(hparams['out_dim'], max_radius)

    datamodule = OptionalInputDataModule(dir="/workspace/SMILES_dataset", FP_choice=hparams["FP_choice"], input_src=["HSQC", "oneD_NMR"], fp_loader = fp_loader_configer.fp_loader, batch_size=1, parser_args=hparams)
    datamodule.setup("test")
    # loader_all_inputs, loader_HSQC_H_NMR, loader_HSQC_C_NMR, loader_only_hsqc, loader_only_1d, loader_only_H_NMR, loader_only_C_NMR = datamodule.test_dataloader()
    test_dataloaders = datamodule.test_dataloader()

    import tqdm
    model.setup_ranker()
    model.ranker.data = model.ranker.data.to(model.device)
    

    from collections import defaultdict

    ## with MW
    mean_rank_records = defaultdict(lambda: defaultdict(list))

    for i in range(11):
        if i<7:
            continue
        loader_idx = i
        if i == 7: # standard HSQC only
            loader_idx = 3
        if i == 8: # standard HSQC, H NMR, and C NMR
            loader_idx = 0
        if i == 9: # standard HSQC and H NMR
            loader_idx = 1
        if i == 10: # standard HSQC and C NMR
            loader_idx = 2
            
        for idx, batch in tqdm.tqdm(enumerate(test_dataloaders[loader_idx])):
            batch[0] = batch[0].to(model.device)
            batch[1] = batch[1].to(model.device)
            batch[2] = batch[2].to(model.device)
            
            if i >= 7:
                # last column all zeros
                batch[0][:, :, -1] = 0
            with torch.no_grad():
                result =  model.test_step(batch, 0, dataloader_idx=loader_idx)
            for k,v in result.items():
                
                mean_rank_records[loader_idx_to_name_mapping[i]][k].append(v)
                
                
    for k,v in mean_rank_records.items():
        for k2, v2 in v.items():
            mean_rank_records[k][k2] = np.mean(v2)
            
    return mean_rank_records

In [7]:
three_model_mean_rank_records = []
for path in [
    "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r2_2048_trial_1/checkpoints/epoch=53-step=36612.ckpt",
    "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r2_2048_trial_2/checkpoints/epoch=66-step=45426.ckpt",
    "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/entropy_on_hashes/flexible_models_jittering_flexible_MW_flexible_normal_hsqc/r2_2048_trial_3/checkpoints/epoch=57-step=39324.ckpt"
]:
    mean_rank_records = test_extra_input_types(path)
    three_model_mean_rank_records.append(mean_rank_records)

Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/vanilla_fp_rankingset_max_radius_2_fp_dim_2048_stacked_together_sparse/FP.pt


4056it [01:02, 65.07it/s]
4056it [01:02, 64.85it/s]
4056it [01:14, 54.72it/s]
4056it [01:20, 50.53it/s]
Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Morgan_FP_loader is already setup
loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/vanilla_fp_rankingset_max_radius_2_fp_dim_2048_stacked_together_sparse/FP.pt


4056it [01:18, 51.59it/s]
4056it [01:07, 60.21it/s]
4056it [01:02, 64.96it/s]
4056it [01:02, 65.09it/s]
Initialized SignCoordinateEncoder[784] with dims [365, 365, 54] and 2 positional encoders. 54 bits are reserved for encoding the final bit


HsqcRankedTransformer saving args
Morgan_FP_loader is already setup
loading /root/gurusmart/MorganFP_prediction/inference_data/inference_rankingset_with_stable_sort/vanilla_fp_rankingset_max_radius_2_fp_dim_2048_stacked_together_sparse/FP.pt


4056it [01:18, 51.49it/s]
4056it [01:20, 50.63it/s]
4056it [01:20, 50.37it/s]
4056it [01:19, 50.83it/s]


In [8]:
from collections import defaultdict
import numpy as np

# Assume your data is stored in three_model_mean_rank_records
# Initialize final result dictionary
average_metrics = defaultdict(dict)

# Get all dataset names (e.g., 'standard HSQC, H NMR, and C NMR', etc.)
dataset_names = three_model_mean_rank_records[0].keys()

for dataset in dataset_names:
    # For each metric under this dataset
    metrics = three_model_mean_rank_records[0][dataset].keys()
    
    for metric in metrics:
        # Collect metric value from all 3 models
        values = [model[dataset][metric] for model in three_model_mean_rank_records]
        average_metrics[dataset][metric] = (float(np.mean(values)), float(np.std(values)))

# Print or use `average_metrics` as needed
for dataset, metrics in average_metrics.items():
    print(f"\n{dataset}")
    for metric_name, avg_val in metrics.items():
        if "rank" in metric_name:
            print(f"  {metric_name}: {avg_val[0]:.4f} ± {avg_val[1]:.4f}")



Normal HSQC only
  mean_rank: 1421.0954 ± 23.1119
  rank_1: 0.3356 ± 0.0094
  rank_5: 0.6724 ± 0.0112
  rank_10: 0.7657 ± 0.0090

standard HSQC, H NMR, and C NMR
  mean_rank: 602.3429 ± 21.6337
  rank_1: 0.3989 ± 0.0013
  rank_5: 0.7473 ± 0.0053
  rank_10: 0.8375 ± 0.0054

standard HSQC and H NMR
  mean_rank: 1148.0311 ± 26.9576
  rank_1: 0.3452 ± 0.0050
  rank_5: 0.6924 ± 0.0129
  rank_10: 0.7893 ± 0.0071

standard HSQC and C NMR
  mean_rank: 598.9983 ± 60.8769
  rank_1: 0.3928 ± 0.0009
  rank_5: 0.7447 ± 0.0073
  rank_10: 0.8332 ± 0.0055


In [9]:
save_path = "/root/gurusmart/MorganFP_prediction/reproduce_previous_works/Spectre/inference/flexible_models_avg_by_3_2048.pkl"
import pickle
with open(save_path, 'wb') as f:
    pickle.dump(average_metrics, f)